In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install optuna

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 8.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 112kB 16.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 133kB 17.9MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=f0b41f08595d776f8bf1f65afc145248fab1679b99645ffc177b235d914735f4
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312964 sha256=4d6a999c1df57647fde7fa4c611cce1a3c48344b3064d1d34e1ac060e16d0beb
  Stored in directory: /root/.cache/pip/wheels/b5/

In [90]:
import os
import random
import re
import warnings

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import pandas_profiling as pdp
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
from IPython.display import HTML
from optuna.integration import lightgbm as lightgbm_opt
from scipy.optimize import minimize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, PReLU, ReLU
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tqdm.notebook import tqdm_notebook

%matplotlib inline
sns.set()
plt.rcParams["figure.figsize"] = (16, 8)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

warnings.simplefilter("ignore")

In [ ]:
path = "drive/My Drive/"
application_train = pd.read_csv(path + "application_train.csv")
application_test = pd.read_csv(path + "application_test.csv")
bureau = pd.read_csv(path + "bureau.csv")

In [ ]:
print('訓練データのデータ数は{}、変数は{}種類です。'.format(application_train.shape[0], application_train.shape[1]))
print('テストデータのデータ数は{}、変数は{}種類です'.format(application_test.shape[0], application_test.shape[1]))

訓練データのデータ数は307511、変数は122種類です。
テストデータのデータ数は48744、変数は121種類です


In [ ]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married

In [ ]:
#  メモ

# dummy_columns = [
#     "NAME_CONTRACT_TYPE",
#     "CODE_GENDER",
#     "FLAG_OWN_CAR",
#     "FLAG_OWN_REALTY",
#     "NAME_TYPE_SUITE",
#     "NAME_INCOME_TYPE",
#     "NAME_EDUCATION_TYPE",
#     "NAME_FAMILY_STATUS",
#     "NAME_HOUSING_TYPE",
#     "OCCUPATION_TYPE",
#     "WEEKDAY_APPR_PROCESS_START",
#     "ORGANIZATION_TYPE",
#     "FONDKAPREMONT_MODE",
#     "HOUSETYPE_MODE",
#     "WALLSMATERIAL_MODE",
#     "EMERGENCYSTATE_MODE"
# ]

# dummied_data = pd.get_dummies(data, columns=dummy_columns, dummy_na=True)

In [ ]:
application_train.isnull().sum()

SK_ID_CURR                           0
TARGET                               0
NAME_CONTRACT_TYPE                   0
CODE_GENDER                          0
FLAG_OWN_CAR                         0
FLAG_OWN_REALTY                      0
CNT_CHILDREN                         0
AMT_INCOME_TOTAL                     0
AMT_CREDIT                           0
AMT_ANNUITY                         12
AMT_GOODS_PRICE                    278
NAME_TYPE_SUITE                   1292
NAME_INCOME_TYPE                     0
NAME_EDUCATION_TYPE                  0
NAME_FAMILY_STATUS                   0
NAME_HOUSING_TYPE                    0
REGION_POPULATION_RELATIVE           0
DAYS_BIRTH                           0
DAYS_EMPLOYED                        0
DAYS_REGISTRATION                    0
DAYS_ID_PUBLISH                      0
OWN_CAR_AGE                     202929
FLAG_MOBIL                           0
FLAG_EMP_PHONE                       0
FLAG_WORK_PHONE                      0
FLAG_CONT_MOBILE         

In [ ]:
# 前処理
amt_annuity = pd.concat([application_train["AMT_ANNUITY"], application_test["AMT_ANNUITY"]])
application_train["AMT_ANNUITY"].fillna(amt_annuity.mean(), inplace=True)
application_test["AMT_ANNUITY"].fillna(amt_annuity.mean(), inplace=True)

amt_goods_price = pd.concat([application_train["AMT_GOODS_PRICE"], application_test["AMT_GOODS_PRICE"]])
application_train["AMT_GOODS_PRICE"].fillna(amt_goods_price.mean(), inplace=True)
application_test["AMT_GOODS_PRICE"].fillna(amt_goods_price.mean(), inplace=True)

own_car_age = pd.concat([application_train["OWN_CAR_AGE"], application_test["OWN_CAR_AGE"]])
application_train["OWN_CAR_AGE"].fillna(own_car_age.mean(), inplace=True)
application_test["OWN_CAR_AGE"].fillna(own_car_age.mean(), inplace=True)

obs_30_cnt_social_circle = pd.concat([application_train["OBS_30_CNT_SOCIAL_CIRCLE"], application_test["OBS_30_CNT_SOCIAL_CIRCLE"]])
application_train["OBS_30_CNT_SOCIAL_CIRCLE"].fillna(obs_30_cnt_social_circle.mean(), inplace=True)
application_test["OBS_30_CNT_SOCIAL_CIRCLE"].fillna(obs_30_cnt_social_circle.mean(), inplace=True)

def_30_cnt_social_circle = pd.concat([application_train["DEF_30_CNT_SOCIAL_CIRCLE"], application_test["DEF_30_CNT_SOCIAL_CIRCLE"]])
application_train["DEF_30_CNT_SOCIAL_CIRCLE"].fillna(def_30_cnt_social_circle.mean(), inplace=True)
application_test["DEF_30_CNT_SOCIAL_CIRCLE"].fillna(def_30_cnt_social_circle.mean(), inplace=True)

obs_60_cnt_social_circle = pd.concat([application_train["OBS_60_CNT_SOCIAL_CIRCLE"], application_test["OBS_60_CNT_SOCIAL_CIRCLE"]])
application_train["OBS_60_CNT_SOCIAL_CIRCLE"].fillna(obs_60_cnt_social_circle.mean(), inplace=True)
application_test["OBS_60_CNT_SOCIAL_CIRCLE"].fillna(obs_60_cnt_social_circle.mean(), inplace=True)

def_60_cnt_social_circle = pd.concat([application_train["DEF_60_CNT_SOCIAL_CIRCLE"], application_test["DEF_60_CNT_SOCIAL_CIRCLE"]])
application_train["DEF_60_CNT_SOCIAL_CIRCLE"].fillna(def_60_cnt_social_circle.mean(), inplace=True)
application_test["DEF_60_CNT_SOCIAL_CIRCLE"].fillna(def_60_cnt_social_circle.mean(), inplace=True)

amt_req_credit_bureau_hour = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_HOUR"], application_test["AMT_REQ_CREDIT_BUREAU_HOUR"]])
application_train["AMT_REQ_CREDIT_BUREAU_HOUR"].fillna(amt_req_credit_bureau_hour.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_HOUR"].fillna(amt_req_credit_bureau_hour.mean(), inplace=True)

amt_req_dredit_bureau_day = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_DAY"], application_test["AMT_REQ_CREDIT_BUREAU_DAY"]])
application_train["AMT_REQ_CREDIT_BUREAU_DAY"].fillna(amt_req_dredit_bureau_day.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_DAY"].fillna(amt_req_dredit_bureau_day.mean(), inplace=True)

amt_req_credit_bureau_week = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_WEEK"], application_test["AMT_REQ_CREDIT_BUREAU_WEEK"]])
application_train["AMT_REQ_CREDIT_BUREAU_WEEK"].fillna(amt_req_credit_bureau_week.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_WEEK"].fillna(amt_req_credit_bureau_week.mean(), inplace=True)

amt_req_credit_bureau_mon = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_MON"], application_test["AMT_REQ_CREDIT_BUREAU_MON"]])
application_train["AMT_REQ_CREDIT_BUREAU_MON"].fillna(amt_req_credit_bureau_mon.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_MON"].fillna(amt_req_credit_bureau_mon.mean(), inplace=True)

amt_req_credit_bureau_qrt = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_QRT"], application_test["AMT_REQ_CREDIT_BUREAU_QRT"]])
application_train["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(amt_req_credit_bureau_qrt.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(amt_req_credit_bureau_qrt.mean(), inplace=True)

amt_req_credit_bureau_year = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_YEAR"], application_test["AMT_REQ_CREDIT_BUREAU_YEAR"]])
application_train["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(amt_req_credit_bureau_year.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(amt_req_credit_bureau_year.mean(), inplace=True)

amt_req_credit_bureau_year = pd.concat([application_train["AMT_REQ_CREDIT_BUREAU_YEAR"], application_test["AMT_REQ_CREDIT_BUREAU_YEAR"]])
application_train["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(amt_req_credit_bureau_year.mean(), inplace=True)
application_test["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(amt_req_credit_bureau_year.mean(), inplace=True)

days_last_phone_change = pd.concat([application_train["DAYS_LAST_PHONE_CHANGE"], application_test["DAYS_LAST_PHONE_CHANGE"]])
application_train["DAYS_LAST_PHONE_CHANGE"].fillna(days_last_phone_change.mean(), inplace=True)
application_test["DAYS_LAST_PHONE_CHANGE"].fillna(days_last_phone_change.mean(), inplace=True)

ext_source_1 = pd.concat([application_train["EXT_SOURCE_1"], application_test["EXT_SOURCE_1"]])
ext_source_2 = pd.concat([application_train["EXT_SOURCE_2"], application_test["EXT_SOURCE_2"]])
ext_source_3 = pd.concat([application_train["EXT_SOURCE_3"], application_test["EXT_SOURCE_3"]])
application_train["EXT_SOURCE_1"].fillna(ext_source_1.mean(), inplace=True)
application_train["EXT_SOURCE_2"].fillna(ext_source_2.mean(), inplace=True)
application_train["EXT_SOURCE_3"].fillna(ext_source_3.mean(), inplace=True)

In [ ]:
application_train.drop('APARTMENTS_AVG', axis=1, inplace=True)
application_train.drop('BASEMENTAREA_AVG', axis=1, inplace=True)
application_train.drop('YEARS_BEGINEXPLUATATION_AVG', axis=1, inplace=True)
application_train.drop('YEARS_BUILD_AVG', axis=1, inplace=True)
application_train.drop('COMMONAREA_AVG', axis=1, inplace=True)
application_train.drop('ELEVATORS_AVG', axis=1, inplace=True)
application_train.drop('ENTRANCES_AVG', axis=1, inplace=True)
application_train.drop('FLOORSMAX_AVG', axis=1, inplace=True)
application_train.drop('FLOORSMIN_AVG', axis=1, inplace=True)
application_train.drop('LANDAREA_AVG', axis=1, inplace=True)
application_train.drop('LIVINGAPARTMENTS_AVG', axis=1, inplace=True)
application_train.drop('LIVINGAREA_AVG', axis=1, inplace=True)
application_train.drop('NONLIVINGAPARTMENTS_AVG', axis=1, inplace=True)
application_train.drop('NONLIVINGAREA_AVG', axis=1, inplace=True)
application_train.drop('APARTMENTS_MODE', axis=1, inplace=True)
application_train.drop('BASEMENTAREA_MODE', axis=1, inplace=True)
application_train.drop('YEARS_BEGINEXPLUATATION_MODE', axis=1, inplace=True)
application_train.drop('YEARS_BUILD_MODE', axis=1, inplace=True)
application_train.drop('COMMONAREA_MODE', axis=1, inplace=True)
application_train.drop('ELEVATORS_MODE', axis=1, inplace=True)
application_train.drop('ENTRANCES_MODE', axis=1, inplace=True)
application_train.drop('FLOORSMAX_MODE', axis=1, inplace=True)
application_train.drop('FLOORSMIN_MODE', axis=1, inplace=True)
application_train.drop('LANDAREA_MODE', axis=1, inplace=True)
application_train.drop('LIVINGAPARTMENTS_MODE', axis=1, inplace=True)
application_train.drop('LIVINGAREA_MODE', axis=1, inplace=True)
application_train.drop('NONLIVINGAPARTMENTS_MODE', axis=1, inplace=True)
application_train.drop('NONLIVINGAREA_MODE', axis=1, inplace=True)
application_train.drop('APARTMENTS_MEDI', axis=1, inplace=True)
application_train.drop('BASEMENTAREA_MEDI', axis=1, inplace=True)
application_train.drop('YEARS_BEGINEXPLUATATION_MEDI', axis=1, inplace=True)
application_train.drop('YEARS_BUILD_MEDI', axis=1, inplace=True)
application_train.drop('COMMONAREA_MEDI', axis=1, inplace=True)
application_train.drop('ELEVATORS_MEDI', axis=1, inplace=True)
application_train.drop('ENTRANCES_MEDI', axis=1, inplace=True)
application_train.drop('FLOORSMAX_MEDI', axis=1, inplace=True)
application_train.drop('FLOORSMIN_MEDI', axis=1, inplace=True)
application_train.drop('LANDAREA_MEDI', axis=1, inplace=True)
application_train.drop('LIVINGAPARTMENTS_MEDI', axis=1, inplace=True)
application_train.drop('LIVINGAREA_MEDI', axis=1, inplace=True)
application_train.drop('NONLIVINGAPARTMENTS_MEDI', axis=1, inplace=True)
application_train.drop('NONLIVINGAREA_MEDI', axis=1, inplace=True)
application_train.drop('FONDKAPREMONT_MODE', axis=1, inplace=True)
application_train.drop('HOUSETYPE_MODE', axis=1, inplace=True)
application_train.drop('TOTALAREA_MODE', axis=1, inplace=True)
application_train.drop('WALLSMATERIAL_MODE', axis=1, inplace=True)
application_train.drop('EMERGENCYSTATE_MODE', axis=1, inplace=True)

In [ ]:
# # OHE
# dummy_columns = [
#     "NAME_CONTRACT_TYPE",
#     "CODE_GENDER",
#     "FLAG_OWN_CAR",
#     "FLAG_OWN_REALTY",
#     "NAME_TYPE_SUITE",
#     "NAME_INCOME_TYPE",
#     "NAME_EDUCATION_TYPE",
#     "NAME_FAMILY_STATUS",
#     "NAME_HOUSING_TYPE",
#     "OCCUPATION_TYPE",
#     "WEEKDAY_APPR_PROCESS_START",
#     "ORGANIZATION_TYPE",
#     "FONDKAPREMONT_MODE",
#     "HOUSETYPE_MODE",
#     "WALLSMATERIAL_MODE",
#     "EMERGENCYSTATE_MODE"
# ]

# dummied_data = pd.get_dummies(application_train, columns=dummy_columns, dummy_na=True)

In [ ]:
application_train.isnull().sum()

SK_ID_CURR                         0
TARGET                             0
NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
CNT_CHILDREN                       0
AMT_INCOME_TOTAL                   0
AMT_CREDIT                         0
AMT_ANNUITY                        0
AMT_GOODS_PRICE                    0
NAME_TYPE_SUITE                 1292
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
REGION_POPULATION_RELATIVE         0
DAYS_BIRTH                         0
DAYS_EMPLOYED                      0
DAYS_REGISTRATION                  0
DAYS_ID_PUBLISH                    0
OWN_CAR_AGE                        0
FLAG_MOBIL                         0
FLAG_EMP_PHONE                     0
FLAG_WORK_PHONE                    0
FLAG_CONT_MOBILE                   0
FLAG_PHONE                         0
F

In [ ]:
print('訓練データのデータ数は{}、変数は{}種類です。'.format(application_train.shape[0], application_train.shape[1]))

訓練データのデータ数は307511、変数は75種類です。


In [ ]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,12.023741,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,12.023741,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,0.509350,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.000000,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,0.501965,0.555912,0.729567,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,12.023741,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,0.501965,0.650442,0.509350,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.005808,0.006281,0.029995,0.231697,0.304399,1.911564
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,12.023741,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,Religion,0.501965,0.322738,0.509350,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
dummy_columns = [
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "OCCUPATION_TYPE",
    "WEEKDAY_APPR_PROCESS_START",
    "ORGANIZATION_TYPE"
]

dummied_data = pd.get_dummies(application_train, columns=dummy_columns, dummy_na=True)

In [ ]:
dummied_data.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_nan,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,CODE_GENDER_nan,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_CAR_nan,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,FLAG_OWN_REALTY_nan,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_nan,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_nan,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_nan,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_nan,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,OCCUPATION_TYPE_nan,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_nan,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Indus

In [ ]:
dummied_data.isnull().sum()

SK_ID_CURR                                           0
TARGET                                               0
CNT_CHILDREN                                         0
AMT_INCOME_TOTAL                                     0
AMT_CREDIT                                           0
AMT_ANNUITY                                          0
AMT_GOODS_PRICE                                      0
REGION_POPULATION_RELATIVE                           0
DAYS_BIRTH                                           0
DAYS_EMPLOYED                                        0
DAYS_REGISTRATION                                    0
DAYS_ID_PUBLISH                                      0
OWN_CAR_AGE                                          0
FLAG_MOBIL                                           0
FLAG_EMP_PHONE                                       0
FLAG_WORK_PHONE                                      0
FLAG_CONT_MOBILE                                     0
FLAG_PHONE                                           0
FLAG_EMAIL

In [ ]:
cnt_fam_members = pd.concat([application_train["CNT_FAM_MEMBERS"], application_test["CNT_FAM_MEMBERS"]])
dummied_data["CNT_FAM_MEMBERS"].fillna(cnt_fam_members.mean(), inplace=True)
application_test["CNT_FAM_MEMBERS"].fillna(cnt_fam_members.mean(), inplace=True)

In [ ]:
# ベースラインモデルの作成
X = dummied_data.iloc[:, 2:].values
y = dummied_data.iloc[:, 1].values

X_test = application_test.iloc[:, 1:].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [87]:
# ランダムフォレスト
rfc = RandomForestClassifier(max_depth=10, min_samples_leaf=1, n_estimators=100, n_jobs=-1, random_state=42)
rfc.fit(X_train, y_train)

print('Train Score: {}'.format(round(rfc.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(rfc.score(X_valid, y_valid), 3)))

Train Score: 0.919
 Test Score: 0.92


In [88]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

print('Logistic Regression \n')
print('Train Score: {}'.format(round(lr.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(lr.score(X_valid, y_valid), 3)))

Logistic Regression 

Train Score: 0.919
 Test Score: 0.92


In [91]:
mlpc = MLPClassifier(hidden_layer_sizes=(100, 100, 10), random_state=0)
mlpc.fit(X_train, y_train)

print('Multilayer Perceptron \n')
print('Train Score: {}'.format(round(mlpc.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(mlpc.score(X_valid, y_valid), 3)))

Multilayer Perceptron 

Train Score: 0.919
 Test Score: 0.92


In [103]:
rfc_pred = rfc.predict_proba(X_valid)
lr_pred = lr.predict_proba(X_valid)
mlpc_pred = mlpc.predict_proba(X_valid)

pred_proba = (rfc_pred + lr_pred + mlpc_pred) / 3
pred = pred_proba.argmax(axis=1)

(array([     0,      1,      2, ..., 215254, 215255, 215256]),)
